In [13]:
import numpy as np
from sklearn.datasets import make_classification, load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split
import torch

from sklearn.model_selection import train_test_split

class ConformalPredictor:


    def __init__(self, model, alpha=0.05):
        self.model = model
        self.alpha = 0.05

    def fit(self, X, y, cal_size=0.33,**kwargs):
        X_train, X_cal, y_train, y_cal = train_test_split(X,y,test_size=cal_size)
        self.model.fit(X_train, y_train,**kwargs)
        y_pred_cal = self.model.predict_proba(X_cal)
        self.scores = 1 - y_pred_cal[np.arange(len(y_cal)), y_cal]
        n = len(self.scores)
        self.threshold = np.quantile(self.scores, np.ceil((n+1)*(1-self.alpha))/n, method="inverted_cdf")

    def predict_set(self,X):
        y_probas = self.model.predict_proba(X)
        pred_sets = []
        for y_proba in y_probas:
            pred_set = np.where(1 - y_proba <= self.threshold)[0]
            pred_sets.append(pred_set)
        return pred_sets

In [14]:
X, y = make_classification(n_samples=500, n_informative = 7, n_classes=3, n_features=10)
# X,y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [17]:
from simple_model import ClassifierModel, DyadRankingModel
from sklearn.linear_model import LogisticRegression


clf = ClassifierModel(input_dim = X_train.shape[1], hidden_dim=16, output_dim=y.max()+1)
rnk = DyadRankingModel(input_dim = X_train.shape[1] + y.max()+1, hidden_dim=16)


cp = ConformalPredictor(clf, alpha=0.1)

cp.fit(X_train, y_train, num_epochs=1000, learning_rate=0.01)

pred_sets = cp.predict_set(X_test)

coverage = np.mean([y_test[i] in pred_sets[i] for i in range(len(y_test))])
efficiency = np.mean([len(pred_sets[i]) for i in range(len(y_test))])

# clf.fit(X_train,y_train, num_epochs=1000, batch_size=1, learning_rate=0.001)
# rnk.fit(X_train,y_train, num_epochs=1000, batch_size=1, learning_rate=0.001)

In [18]:
print(coverage)
print(efficiency)

0.9696969696969697
1.8181818181818181
